In [1]:
import pandas as pd
import yfinance as yf
from yahoo_fin import stock_info as si
from sqlalchemy import create_engine
import urllib
import time
from datetime import datetime
from datetime import date
import datetime

#### **Отримуємо список тікерів, що входять до індексу SP100**

In [2]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [18]:
engine.execute('''
                update [dbo].[tb_sp100_tickers]
                set [Ticker] = 'BRK-B'
                from [dbo].[tb_sp100_tickers]
                where [Ticker] = 'BRKB';
                ''')

In [19]:
result = engine.execute('SELECT [Ticker] FROM [dbo].[tb_sp100_tickers]')
df_SP100 = pd.DataFrame(result.fetchall())
df_SP100

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,ADBE
...,...
96,V
97,VZ
98,WFC
99,WMT


In [20]:
ticker_list = df_SP100[df_SP100.columns[0]].values.tolist()
len(ticker_list)

101

In [21]:
ticker_list[0:5]

['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE']

In [22]:
pd.options.display.max_colwidth = 5000

#### **Отримуємо інформацію про компанію та дані по виплатах дивідендів для всіх акцій з SP100**

In [10]:
!pip install --upgrade yfinance


[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 2.9/2.9 MB 370.8 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 69.0/69.0 kB 539.2 kB/s eta 0:00:00
   ---------------------------------------- 62.6/62.6 kB 556.7 kB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.0-py3-none-any.whl size=135766 sha256=f391f9f457a4ab5a17f99ecabdd4f0c564f530e45d28b70b5a72fd6eca93f65d
  Stored in directory: c:\users\51fb~1\appdata\local\pip\cache\wheels\c2\ce\84\33a31720d239d310cfff49d7bf56da412b6caefec2e112f327
Successfully built peewee
  Attempting uninstall: requests
    Found existing installation: requests 2.28.2
    Uninstalling requests-2.28.2:
     

In [23]:
df_stock_final = pd.DataFrame()
df_dividend_final = pd.DataFrame()
i = 1

for ticker in ticker_list:
  stock = yf.Ticker(ticker)

  df_dividends = stock.dividends.to_frame().reset_index()

  if df_dividends.shape[0] > 0:
    df_dividends['Ticker'] = ticker
    df_dividend_final=pd.concat([df_dividend_final, df_dividends], ignore_index=True)
  
  print(str(i) + '. '+ ticker + '. Dividend data (rows): ' + str(df_dividends.shape[0]) + '.')
  i += 1

1. AAPL. Dividend data (rows): 81.
2. ABBV. Dividend data (rows): 44.
3. ABT. Dividend data (rows): 164.
4. ACN. Dividend data (rows): 41.
5. ADBE. Dividend data (rows): 63.
6. AIG. Dividend data (rows): 140.
7. AMD. Dividend data (rows): 0.
8. AMGN. Dividend data (rows): 50.
9. AMT. Dividend data (rows): 49.
10. AMZN. Dividend data (rows): 0.
11. AVGO. Dividend data (rows): 53.
12. AXP. Dividend data (rows): 188.
13. BA. Dividend data (rows): 228.
14. BAC. Dividend data (rows): 151.
15. BK. Dividend data (rows): 146.
16. BKNG. Dividend data (rows): 0.
17. BLK. Dividend data (rows): 82.
18. BMY. Dividend data (rows): 205.
19. BRK-B. Dividend data (rows): 0.
20. C. Dividend data (rows): 140.
21. CAT. Dividend data (rows): 199.
22. CHTR. Dividend data (rows): 0.
23. CL. Dividend data (rows): 202.
24. CMCSA. Dividend data (rows): 110.
25. COF. Dividend data (rows): 115.
26. COP. Dividend data (rows): 176.
27. COST. Dividend data (rows): 84.
28. CRM. Dividend data (rows): 0.
29. CSCO. Divi

In [24]:
df_dividend_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12557 entries, 0 to 12556
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype                           
---  ------     --------------  -----                           
 0   Date       12557 non-null  datetime64[ns, America/New_York]
 1   Dividends  12557 non-null  float64                         
 2   Ticker     12557 non-null  object                          
dtypes: datetime64[ns, America/New_York](1), float64(1), object(1)
memory usage: 294.4+ KB


In [25]:
df_dividend_final

,Date,Dividends,Ticker
0,1987-05-11 00:00:00-04:00,0.000536,AAPL
1,1987-08-10 00:00:00-04:00,0.000536,AAPL
2,1987-11-17 00:00:00-05:00,0.000714,AAPL
3,1988-02-12 00:00:00-05:00,0.000714,AAPL
4,1988-05-16 00:00:00-04:00,0.000714,AAPL
...,...,...,...
12552,2022-11-14 00:00:00-05:00,0.910000,XOM
12553,2023-02-13 00:00:00-05:00,0.910000,XOM
12554,2023-05-15 00:00:00-04:00,0.910000,XOM
12555,2023-08-15 00:00:00-04:00,0.910000,XOM


In [26]:
df_dividend_final['Date'] = pd.to_datetime( df_dividend_final['Date'], errors='coerce',utc=True)

In [27]:
df_dividend_final['Date'] = df_dividend_final['Date'].dt.date

In [28]:
df_dividend_final

,Date,Dividends,Ticker
0,1987-05-11,0.000536,AAPL
1,1987-08-10,0.000536,AAPL
2,1987-11-17,0.000714,AAPL
3,1988-02-12,0.000714,AAPL
4,1988-05-16,0.000714,AAPL
...,...,...,...
12552,2022-11-14,0.910000,XOM
12553,2023-02-13,0.910000,XOM
12554,2023-05-15,0.910000,XOM
12555,2023-08-15,0.910000,XOM


In [29]:
df_dividend_final["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
df_dividend_final["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
df_dividend_final

,Date,Dividends,Ticker,date_insert,time_insert
0,1987-05-11,0.000536,AAPL,2024-01-02,18:15:06
1,1987-08-10,0.000536,AAPL,2024-01-02,18:15:06
2,1987-11-17,0.000714,AAPL,2024-01-02,18:15:06
3,1988-02-12,0.000714,AAPL,2024-01-02,18:15:06
4,1988-05-16,0.000714,AAPL,2024-01-02,18:15:06
...,...,...,...,...,...
12552,2022-11-14,0.910000,XOM,2024-01-02,18:15:06
12553,2023-02-13,0.910000,XOM,2024-01-02,18:15:06
12554,2023-05-15,0.910000,XOM,2024-01-02,18:15:06
12555,2023-08-15,0.910000,XOM,2024-01-02,18:15:06


In [30]:
df_dividend_final.to_sql('tb_dividends', schema='dbo', con = engine, if_exists='append',index=False)

In [31]:
result = engine.execute('SELECT COUNT(*) FROM [dbo].[tb_dividends]')
result.fetchall()[0][0] 

58892

In [35]:
result = engine.execute('SELECT count(*) FROM [DividendPolicyResearch].[dbo].[tb_dividends] where year(date_insert) = 2024')
result.fetchall()[0][0] 

12557